Image Segmentation use: https://docs.ultralytics.com/modes/predict/#inference-sources

For model Training use: https://github.com/computervisioneng/image-classification-yolov8/tree/main/local_env

Hyper parameter tuning: https://docs.ultralytics.com/guides/hyperparameter-tuning/


## Importing Libraries

Importing the necessary libraries for the project

And initializing constants

```python

```


In [3]:
%pip install opencv-python-headless numpy ultralytics Pillow

Note: you may need to restart the kernel to use updated packages.


You should consider upgrading via the 'c:\Users\domai\AppData\Local\Programs\Python\Python38\python.exe -m pip install --upgrade pip' command.


In [2]:
import cv2
import numpy as np
import os
from ultralytics import YOLO
import pandas as pd
import matplotlib.pyplot as plt
from PIL import Image, ImageEnhance, ImageOps

In [103]:
### CONSTANTS
# Image properties
IMAGE_WIDTH = 640
IMAGE_HEIGHT = 480
IMAGE_CHANNELS = 3

IMAGE_SIZE = (IMAGE_WIDTH, IMAGE_HEIGHT)

IMAGE_EXT = ('.jpg', '.jpeg', '.png', '.bmp', '.tif', '.tiff', '.gif')


# Director paths
PROCESSED_DS_PATH_NM = './processed_dataset/NoMask'
PROCESSED_DS_PATH_M = './processed_dataset/Mask'
RAW_DS_PATH = './Data'

## MODELS
# Segmentation
SEG_KWARGS = {
    'classes': 0,
    'imgsz': IMAGE_SIZE,
    'boxes':True,
    'save':True,
    'show_labels':False,
    'show_conf':True,
    'max_det':1,
}
# Classification
CLASS_KWARGS = {
    'epochs': 20,
    'project': './models',
}
MODEL_KWARGS = {
    'save': True,
    'project': './',
    'name': 'tests',
}

## Preparig data for training


### Preparing Models

```python

```


In [4]:
img_seg_model = YOLO('yolov8n-seg.pt')
img_class_model = YOLO("yolov8n-cls.pt")

100%|██████████| 6.73M/6.73M [00:00<00:00, 28.9MB/s]
100%|██████████| 5.28M/5.28M [00:00<00:00, 31.4MB/s]


In [55]:
with Image.open('processed_dataset\Mask\val\6\sthephany_6_0.jpg') as img:

    # add a border
    img = ImageOps.expand(img,border=50,fill='black')

    # Enhancing the sharpness
    enhancer = ImageEnhance.Sharpness(img)
    enhanced_img = enhancer.enhance(7)
    
    
    # Save it to a new file
    enhanced_img.save('./result.jpg')

OSError: [Errno 22] Invalid argument: 'processed_dataset\\Mask\x0bal\x06\\sthephany_6_0.jpg'

In [85]:
img_seg_model('./result.jpg', **SEG_KWARGS)

FileNotFoundError: ./result.jpg does not exist

### Preparing dataset

```python

```


In [86]:
def process_images(
        source_folder=RAW_DS_PATH, 
        destination_folder=PROCESSED_DS_PATH_NM, 
        grayscale=True, 
        resize=True,
        resize_dimensions=IMAGE_SIZE):
    if not os.path.exists(source_folder):
        print(f"Source folder does not exist: {source_folder}")
        return
    if os.path.exists(destination_folder):
        print(f"Destination folder already exists: {destination_folder}")
        return

    os.makedirs(destination_folder, exist_ok=True)

    for subdir, dirs, files in os.walk(source_folder):
        for file in files:
            if file.lower().endswith(IMAGE_EXT):
                source_path = os.path.join(subdir, file)
                relative_path = os.path.relpath(subdir, source_folder)
                destination_subdir = os.path.join(destination_folder, relative_path)
                os.makedirs(destination_subdir, exist_ok=True)

                destination_path = os.path.join(destination_subdir, file)

                image = cv2.imread(source_path)
                if image is None:
                    print(f"Unable to read image: {source_path}")
                    continue

                if grayscale:
                    image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

                if resize:
                    image = cv2.resize(image, resize_dimensions)

                cv2.imwrite(destination_path, image)

In [123]:
def mask_images(
        source_folder = PROCESSED_DS_PATH_NM,
        destination_folder=PROCESSED_DS_PATH_M):
    
    if not os.path.exists(source_folder):
        print(f"Source folder does not exist: {source_folder}")
        return
    if os.path.exists(destination_folder):
        print(f"Destination folder already exists: {destination_folder}")
        return

    root_path_length = len(source_folder) + len(os.path.sep)

    # Set comprehension to get only directories at depth 2
    subdirs = {subdir[root_path_length:] for subdir, dirs, files in os.walk(source_folder)
               if subdir.count(os.path.sep) == 2}
    
    for subdir in subdirs:
        img_folder = os.path.join(source_folder, subdir)
        print(img_folder)
        img_seg_model(img_folder, project=destination_folder, name=subdir, **SEG_KWARGS)
    

In [88]:
process_images()

In [104]:
SEG_KWARGS

{'classes': 0,
 'imgsz': (640, 480),
 'boxes': True,
 'save': True,
 'show_labels': False,
 'show_conf': True,
 'max_det': 1}

In [105]:
mask_images()

Destination folder already exists: ./processed_dataset/Mask


## Training

```python

```


In [140]:
try:
    run = max(os.listdir(CLASS_KWARGS['project']))
    run = 1 + int(run)
except:
    run = 1

img_class_model.train(data=PROCESSED_DS_PATH_M, name=run, **CLASS_KWARGS)

Ultralytics YOLOv8.0.207  Python-3.8.10 torch-2.1.0+cpu CPU (AMD Ryzen 5 5600 6-Core Processor)
engine\trainer: task=classify, mode=train, model=yolov8n-cls.pt, data=./processed_dataset/Mask, epochs=20, patience=50, batch=16, imgsz=224, save=True, save_period=-1, cache=False, device=None, workers=0, project=./models, name=4, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, show=False, save_txt=False, save_conf=False, save_crop=False, show_labels=True, show_conf=True, vid_stride=1, stream_buffer=False, line_width=None, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, boxes=True, format=torchscript, keras

ultralytics.utils.metrics.ClassifyMetrics object with attributes:

confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x00000271E3F03310>
curves: []
curves_results: []
fitness: 0.757709264755249
keys: ['metrics/accuracy_top1', 'metrics/accuracy_top5']
results_dict: {'metrics/accuracy_top1': 0.5286343693733215, 'metrics/accuracy_top5': 0.9867841601371765, 'fitness': 0.757709264755249}
save_dir: WindowsPath('models/4')
speed: {'preprocess': 0.0, 'inference': 3.3876591316928946, 'loss': 0.0, 'postprocess': 0.0}
task: 'classify'
top1: 0.5286343693733215
top5: 0.9867841601371765

## Performance Evaluation

In [ ]:
cm_path = os.path.join(CLASS_KWARGS['project'], str(run), 'confusion_matrix.jpg')
results_path = os.path.join(CLASS_KWARGS['project'], str(run), 'results.jpg')

cm_img = plt.imread(cm_path)
results_img = plt.imread(results_path)

plt.imshow(cm_img)
plt.show()

plt.imshow(results_img)
plt.show()

## Prediction


In [25]:
def get_model(run, best=False):
    weight = 'weights/best.pt' if best else 'weights/last.pt'
    path = os.path.join(CLASS_KWARGS['project'], str(run), weight)
    return YOLO(path)

In [51]:
def predict(image_path):
    try:
        runs = os.listdir(CLASS_KWARGS['project'])
    except:
        print("No runs found")
        return
    i = input(f"Choose a run from {runs}: ")

    while i not in runs:
        i = input(f"Choose a run from {runs}: ")
    
    model = get_model(i)

    model(image_path, **MODEL_KWARGS)

In [143]:

predict('./Data/datatest_processed_masked/datat/datat2/')




image 1/41 c:\Users\domai\Desktop\PFJUPITER\driver_detection\Data\datatest_processed_masked\datat\datat2\0151_video1_jpg.rf.1935373158fdb7cadb1f7514f67ab487.jpg: 224x224 2 1.00, 5 0.00, 4 0.00, 3 0.00, 0 0.00, 10.0ms
image 2/41 c:\Users\domai\Desktop\PFJUPITER\driver_detection\Data\datatest_processed_masked\datat\datat2\0283_video1_jpg.rf.f5a2078cff98647a810ee39e276ef13a.jpg: 224x224 2 1.00, 5 0.00, 0 0.00, 1 0.00, 4 0.00, 10.0ms
image 3/41 c:\Users\domai\Desktop\PFJUPITER\driver_detection\Data\datatest_processed_masked\datat\datat2\1239_video1_jpg.rf.9ceaa7e5dc3fe6c5563f79e71574c18d.jpg: 224x224 4 0.47, 1 0.14, 5 0.13, 0 0.09, 3 0.09, 9.0ms
image 4/41 c:\Users\domai\Desktop\PFJUPITER\driver_detection\Data\datatest_processed_masked\datat\datat2\img_0_jpg.rf.de2446ced26aa599ff593cd179da222a.jpg: 224x224 4 1.00, 2 0.00, 0 0.00, 5 0.00, 1 0.00, 10.0ms
image 5/41 c:\Users\domai\Desktop\PFJUPITER\driver_detection\Data\datatest_processed_masked\datat\datat2\img_1207_jpg.rf.74483f16b9e4fac1b

In [147]:
#process_images(source_folder='./Data/datanew', destination_folder='./Data/datanewresult/')
mask_images(source_folder='./Data/datanewresult', destination_folder='./Data/datanewresultmask/')


image 1/70 c:\Users\domai\Desktop\PFJUPITER\driver_detection\Data\datanewresult\data2\adda\0004_video1_jpg.rf.b7d8aa6747750bb315b3e8bd48845096.jpg: 384x480 1 person, 50.0ms
image 2/70 c:\Users\domai\Desktop\PFJUPITER\driver_detection\Data\datanewresult\data2\adda\0025_video2_3105_jpg.rf.68180040115d7828c0ab130dec831ab0.jpg: 384x480 1 person, 44.0ms
image 3/70 c:\Users\domai\Desktop\PFJUPITER\driver_detection\Data\datanewresult\data2\adda\0050_video1_3105_jpg.rf.6cd9ea58c85a6f61f5f88311791a0c55.jpg: 384x480 1 person, 45.0ms


./Data/datanewresult\data2\adda


image 4/70 c:\Users\domai\Desktop\PFJUPITER\driver_detection\Data\datanewresult\data2\adda\0064_video1_jpg.rf.e95cd6edec310dba1a994e8544666669.jpg: 384x480 1 person, 48.0ms
image 5/70 c:\Users\domai\Desktop\PFJUPITER\driver_detection\Data\datanewresult\data2\adda\0086_video1_jpg.rf.38e5240aa5416583be583dd0270ac00d.jpg: 384x480 1 person, 46.0ms
image 6/70 c:\Users\domai\Desktop\PFJUPITER\driver_detection\Data\datanewresult\data2\adda\0096_video3_3105_jpg.rf.1b4659a5e7109c79a20e6cbea63fc7e2.jpg: 384x480 1 person, 44.0ms
image 7/70 c:\Users\domai\Desktop\PFJUPITER\driver_detection\Data\datanewresult\data2\adda\0130_video1_3105_jpg.rf.f5f7cb44b29e72cca76a4742e0303a2a.jpg: 384x480 1 person, 45.0ms
image 8/70 c:\Users\domai\Desktop\PFJUPITER\driver_detection\Data\datanewresult\data2\adda\0132_video1_3105_jpg.rf.65f284688e4d3f772f0a4d40317f0c13.jpg: 384x480 1 person, 47.0ms
image 9/70 c:\Users\domai\Desktop\PFJUPITER\driver_detection\Data\datanewresult\data2\adda\0144_video1_jpg.rf.cbd90701f4